In [1]:
!nvidia-smi

Sat Jan 22 16:23:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 3.3 MB 53.8 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 248 kB 5.1 MB/s 
     |████████████████████████████████| 1.7 MB 70.5 MB/s 
     |████████████████████████████████| 1.2 MB 54.0 MB/s 
     |████████████████████████████████| 9.3 MB 32.7 MB/s 
     |████████████████████████████████| 311 kB 67.1 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 180 kB 64.7 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 142 kB 69.2 MB/s 
     |████████████████████████████████| 63 kB 2

In [3]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import LabelEncoder

In [10]:
data = pd.read_table('/content/B3DB_classification.tsv')
X = data['SMILES']
target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [11]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [12]:
Xtrain = {'text':xtrain, 'labels':ytrain}
Xvalid = {'text':xvalid, 'labels':yvalid}
Xtest = {'text':xtest, 'labels':ytest}
train_df = pd.DataFrame(Xtrain)
valid_df = pd.DataFrame(Xvalid)
test_df = pd.DataFrame(Xtest)

In [14]:
model = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 82,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'early_stopping_delta': 0.01,
'early_stopping_metric':"auroc",
'early_stopping_metric_minimize' : False,
'early_stopping_patience' : 10,
'max_seq_length' : 128,
'num_train_epochs' : 10,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=2,) # You can set class weights by using the optional weight argument


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE

Downloading:   0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

In [15]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [16]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'bbb_model'})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/5245 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.754061620705592, 'tp': 300, 'tn': 587, 'fp': 50, 'fn': 63, 'auroc': 0.9446246394298343, 'auprc': 0.9125724870075168, 'eval_loss': 0.563843653678894}
INFO:simpletransformers.classification.classification_model: No improvement in auroc
INFO:simpletransformers.classification.classification_model: Current step: 10
INFO:simpletransformers.classification.classification_model: Early stopping patience: 10
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.7522020269023171, 'tp': 301, 'tn': 585, 'fp': 52, 'fn': 62, 'auroc': 0.9424450008865594, 'auprc': 0.9023807682938698, 'eval_loss': 0.5556943477392197}
INFO:simpletransformers.classification.classification_model: Patience of 10 steps reached
INFO:simpletransformers.classification.classification_model: Training terminated.
INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.


In [17]:

model = ClassificationModel(
    "roberta", "/content/outputs/best_model",args={'wandb_project': 'bbb_model'}
)

In [18]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1562 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/196 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,█▅▆▇▅▆▄▃▃▂▂▂▆▃▁▁▁▂▁▁▁▁
auprc,▁▄▄▇▇▇█▇▇▇██▇▇▇▃▅▅▆▅
auroc,▁▅▅▇▇▇█▇▇███▇▇▇▆▇▇▇▇
eval_loss,▃▂▂▁▁▁▂▂▂▂▃▃▄▅▅▆▇▇██
fn,█▆▆▃▂▂▂▂▂▃▃▃▂▁▁▂▃▃▂▂
fp,▁▂▂▆██▆▄▄▂▂▂▅▇▇▅▂▂▄▄
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇█
lr,▃██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▁▁
mcc,▁▃▃▅▆▆▇▇▇▇▇▇███▇▇▇▇▇
tn,█▇▇▃▁▁▃▅▅▇▇▇▄▂▂▄▇▇▅▅
tp,▁▃▃▆▇▇▇▇▇▆▆▆▇██▇▆▆▇▇


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6908633515809921, 'tp': 458, 'tn': 880, 'fp': 121, 'fn': 103, 'auroc': 0.9316583594658461, 'auprc': 0.8889649260721295, 'acc': 0.8565941101152369, 'eval_loss': 0.3220868074164099}


{'mcc': 0.6908633515809921, 'tp': 458, 'tn': 880, 'fp': 121, 'fn': 103, 'auroc': 0.9316583594658461, 'auprc': 0.8889649260721295, 'acc': 0.8565941101152369, 'eval_loss': 0.3220868074164099}


In [19]:
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_128_2_2


Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.7150802160091179, 'tp': 292, 'tn': 577, 'fp': 60, 'fn': 71, 'auroc': 0.9447176200422954, 'auprc': 0.915724387338566, 'acc': 0.869, 'eval_loss': 0.28680906105041504}


{'mcc': 0.7150802160091179, 'tp': 292, 'tn': 577, 'fp': 60, 'fn': 71, 'auroc': 0.9447176200422954, 'auprc': 0.915724387338566, 'acc': 0.869, 'eval_loss': 0.28680906105041504}


In [20]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,462 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories c

In [21]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [22]:
from transformers import AutoModel
from transformers import AutoTokenizer

model = AutoModel.from_pretrained("/content/outputs/best_model")
model.push_to_hub("BBB_prediction_finetuned_classification")

tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")
tokenizer.push_to_hub("BBB_prediction_finetuned_classification")

Some weights of the model checkpoint at /content/outputs/best_model were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /content/outputs/best_model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cloning ht

Upload file pytorch_model.bin:   0%|          | 3.37k/318M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification
   1dec82d..5006386  main -> main

   1dec82d..5006386  main -> main

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification
   5006386..27bdbb8  main -> main

   5006386..27bdbb8  main -> main



'https://huggingface.co/Parsa/BBB_prediction_finetuned_classification/commit/27bdbb89fafbde7384ddfdf78d34e5b9005910e7'